In [1]:
from openravepy import *
import numpy
env = Environment() # create openrave environment
env.SetViewer('qtcoin') # attach viewer (optional)
RaveSetDebugLevel(DebugLevel.Debug)


True

In [ ]:
env.Load("../../fh_desc/model.xml")
env.Load("data/table_washer.kinbody.xml")

robot = env.GetRobots()[0]
manip = robot.GetActiveManipulator()

ikmodel = databases.inversekinematics.InverseKinematicsModel(robot,iktype=IkParameterization.Type.Transform6D)
if not ikmodel.load():
    ikmodel.autogenerate()

In [ ]:
robot.SetDOFValues([0.0, -0.75, 2.2, 0.5],[0,1,2,3]) # set the first 4 dof values
Tee = manip.GetEndEffectorTransform()
print Tee

In [ ]:
manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
Tgoal = numpy.array([[0.37,0.92,0,0.4],[0,0,1,-0.23],[0.93,-0.37,0,0.12],[0,0,0,1]])
print Tgoal

In [ ]:
res = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10) # call motion planner with goal joint angles
robot.WaitForController(0) # wait

In [ ]:
robot.SetDOFValues([0.0, -2.35, -2.6, 0.5],[0,1,2,3]) # set the first 4 dof values
Tee = manip.GetEndEffectorTransform()
print Tee

In [ ]:
manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
Tgoal = numpy.array([[0.51,0.86,0,-0.4],[0,0,1,-0.23],[0.86,-0.51,0,0.12],[0,0,0,1]])
print Tgoal

In [ ]:
res = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10) # call motion planner with goal joint angles
robot.WaitForController(0) # wait

In [ ]:
for _ in range(10):
    manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
    Tgoal = numpy.array([[0.37,0.92,0,0.4],[0,0,1,-0.23],[0.93,-0.37,0,0.12],[0,0,0,1]])
    res = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10) # call motion planner with goal joint angles
    robot.WaitForController(0) # wait

    manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
    Tgoal = numpy.array([[0.51,0.86,0,-0.4],[0,0,1,-0.23],[0.86,-0.51,0,0.12],[0,0,0,1]])
    res = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10) # call motion planner with goal joint angles
    robot.WaitForController(0) # wait

In [ ]:
env.Load("data/mug1.kinbody.xml")

In [ ]:
mug = env.GetKinBody('mug')
mug_pose = numpy.array([[1,0,0,0],[0,0,-1,0],[0,1,0,0.05],[0,0,0,1]])

mug.SetTransform(mug_pose)

In [ ]:
gmodel = databases.grasping.GraspingModel(robot,mug)
if not gmodel.load():
    gmodel.autogenerate()

In [ ]:
validgrasps, validindicees = gmodel.computeValidGrasps(returnnum=1)
gmodel.moveToPreshape(validgrasps[0])
Tgoal = gmodel.getGlobalGraspTransform(validgrasps[0],collisionfree=True)
basemanip = interfaces.BaseManipulation(robot)
basemanip.MoveToHandPosition(matrices=[Tgoal])
robot.WaitForController(0)
taskmanip = interfaces.TaskManipulation(robot)
taskmanip.CloseFingers()
robot.WaitForController(0)

In [ ]:
import time
for _ in range(2):
    starttime = time.time()
    manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
    Tgoal = numpy.array([[0.37,0.92,0,0.4],[0,0,1,-0.23],[0.93,-0.37,0,0.12],[0,0,0,1]])
    traj = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10l,execute=False,outputtrajobj=True) # call motion planner with goal joint angles
    print('non-inkstatistics planning time: %fs'%(time.time()-starttime))
    robot.GetController().SetPath(traj)
    robot.WaitForController(0) # wait
    
    starttime = time.time()
    manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
    Tgoal = numpy.array([[0.51,0.86,0,-0.4],[0,0,1,-0.23],[0.86,-0.51,0,0.12],[0,0,0,1]])
    traj = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10l,execute=False,outputtrajobj=True) # call motion planner with goal joint angles
    print('non-linkstatistics planning time: %fs'%(time.time()-starttime))
    robot.GetController().SetPath(traj)
    robot.WaitForController(0) # wait
    
lmodel=databases.linkstatistics.LinkStatisticsModel(robot)
if not lmodel.load():
    lmodel.autogenerate()
    
lmodel.setRobotResolutions(0.5) # set resolution given smallest object is 0.1m
lmodel.setRobotWeights() # set the weights for planning
for _ in range(2):
    starttime = time.time()
    manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
    Tgoal = numpy.array([[0.37,0.92,0,0.4],[0,0,1,-0.23],[0.93,-0.37,0,0.12],[0,0,0,1]])
    traj = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10l,execute=False,outputtrajobj=True) # call motion planner with goal joint angles
    print('linkstatistics planning time: %fs'%(time.time()-starttime))
    robot.GetController().SetPath(traj)
    robot.WaitForController(0) # wait
    
    starttime = time.time()
    manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
    Tgoal = numpy.array([[0.51,0.86,0,-0.4],[0,0,1,-0.23],[0.86,-0.51,0,0.12],[0,0,0,1]])
    traj = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10l,execute=False,outputtrajobj=True) # call motion planner with goal joint angles
    print('linkstatistics planning time: %fs'%(time.time()-starttime))
    robot.GetController().SetPath(traj)
    robot.WaitForController(0) # wait
    
#starttime = time.time()
#traj=manipprob.MoveManipulator(goal=goal,execute=False,outputtrajobj=True)
#raveLogInfo('linkstatistics planning time: %fs'%(time.time()-starttime))
#robot.GetController().SetPath(traj)
#robot.WaitForController(0)